In [1]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('customer_churn.db')

In [2]:
# --- SQL to clean and prepare ML-ready dataset ---
query = """
SELECT 
    c.customer_id,
    c.plan_type,
    c.country,
    COUNT(DISTINCT a.event_date) AS active_days,
    SUM(CASE WHEN p.success = 1 THEN 1 ELSE 0 END) AS successful_payments,
    MAX(p.payment_date) AS last_payment_date,
    CASE WHEN ch.customer_id IS NOT NULL THEN 1 ELSE 0 END AS churned
FROM customers c
LEFT JOIN activity_logs a ON c.customer_id = a.customer_id
LEFT JOIN payments p ON c.customer_id = p.customer_id
LEFT JOIN churned_customers ch ON c.customer_id = ch.customer_id
GROUP BY c.customer_id, c.plan_type, c.country, ch.customer_id;
"""

In [3]:
# Execute the SQL query and load into pandas
df_clean = pd.read_sql_query(query, conn)

# Save to CSV for ML step
df_clean.to_csv("cleaned_churn_data.csv", index=False)

conn.close()
print("✅ Cleaned data exported to 'cleaned_churn_data.csv'")

✅ Cleaned data exported to 'cleaned_churn_data.csv'
